In [16]:
import mysql.connector
from mysql.connector import Error
import pywt
import json
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [17]:
connection = mysql.connector.connect(
        user='USER', password='PASSWORD', host='mysql', port='3306', database='SATELLITE_DB'
    )

In [18]:
if connection.is_connected():
    cursor = connection.cursor()
    print("conneted")

id_img = ""

In [19]:
try:
    cursor.execute(f"""SELECT c.codebook, l.label,i.coeff_img
                   FROM infor_images as i
                   inner join codebooks as c on i.id_img = c.id_img
                   inner join labels as l on i.id_img = l.id_img
                   where i.id_image = {id_img} and c.band = "B1" and l.band = "B1";""")
    labels_data = cursor.fetchall()
except Error as e:
    print(f"Error: {e}")

In [20]:
for codebook_json, label_json,coeff_img_json in labels_data:
    codebook = json.loads(codebook_json)
    label = json.loads(label_json)
    coeff = json.loads(coeff_img_json)

In [21]:
wav = "db4"

match_im = np.zeros((label.shape[0],label.shape[1]), dtype=float)
count = 0
# Loop through the loop to reconstruct the quantized image
for i in range(label.shape[0]):
    for j in range(label.shape[1]):
        if label[i][j] != 0:
            match_im[i][j] =codebook[count]
            count+=1
            
coeffs_filt = pywt.array_to_coeffs(match_im, coeff, output_format='wavedec2')
# Regenerate the original image
Arecon = pywt.waverec2(coeffs_filt, wavelet=wav)
Arecon = (Arecon - np.min(Arecon)) / (np.max(Arecon) - np.min(Arecon)) * 255

In [22]:
plt.imshow(Arecon)